In [61]:
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure
import pandas as pd
import numpy as np

def parse_layout(layout, a, b):
    
    textDf = pd.DataFrame(columns=['x1', 'y1', 'x2', 'y2','y','text'])
    
    """Function to recursively parse the layout tree."""
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            for obj in lt_obj:
                s = pd.Series({'x1':int(obj.bbox[0]), 'y1':int(obj.bbox[1]),'x2':int(obj.bbox[2]), 
                               'y2':int(obj.bbox[3]), 'y':float(0), 'text':obj.get_text().replace("\n", '')})
                textDf = textDf.append(s, ignore_index = True)  
        #elif isinstance(lt_obj, LTFigure):
        #    parse_layout(lt_obj, b)  # Recursive
    
    
    textDf = textDf.sort_values(['x1','y1'], ascending=True).reset_index(drop = True)
    
    row_startDf = textDf[textDf['x1'] < textDf['x1'].min(axis=0) + b]
    rows = row_identifier(row_startDf, a)
    
    for i in range(len(textDf)):
        y = -1
        for j in range(len(rows)):
            if float(textDf.iloc[i]['y2']) < rows.iloc[j]['upper_y'] and float(textDf.iloc[i]['y1']) > rows.iloc[j]['lower_y']:
                y = rows.iloc[j]['y']
                break
        #textDf.set_value(i, 'y', y)
        textDf.at[i:i, 'y'] = y
    
    return textDf


def row_identifier(row_startDf, a):
    rows = pd.DataFrame(columns=['lower_y', 'upper_y', 'y'])
    height = row_startDf['y2']-row_startDf['y1']
    rows['upper_y'] = row_startDf['y2'] + height*a
    rows['lower_y'] = row_startDf['y1'] - height*a
    rows['y'] = (rows['upper_y'] + rows['lower_y'])/2
    return rows 
       

def convert_to_df(text):
    
    #courseDf = pd.DataFrame(columns = ('Course', 'Credit points obtained', 'Major(s)'))
    #marksDf = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'))
    
        student = []
        course = []
        mark = []
        indexMark = -1
        indexResultS = -1
        indexResultE = -1
        
        for i in range(len(text)):
            content = text.iloc[i]['text']
            temp = content.split(',')
            if 'Student name' in temp[0]:
                student.append(temp[0][12:].lstrip())
            elif 'Student ID' in temp[0]:
                student.append(temp[0][10:].lstrip())
            elif 'Student course ID' in temp[0]:
                student.append(temp[0][17:].lstrip())
            elif temp[0] == 'Course':
                course.append(temp[1])
            elif len(temp) == 1 and 'Credit points obtained' in temp[0]:
                course.append(temp[0][22:])
            elif len(temp) == 2 and 'Credit points obtained' in temp[0]:
                course.append(temp[1])
            elif len(temp) == 2 and 'Major' in temp[0]:
                course.append(temp[1])
            elif temp:
                
            elif len(temp) == 4 and temp[0] == 'Year':
                indexMark = i
                break
        
        if text.iloc[len(text)-1]['text'] == 'End of course result history':        
            #print("end")
            for i in range(indexMark, len(text)-1):
                content = text.iloc[i]['text']
                temp = content.split(',')
                mark.append(temp)
            mark = [y for x in mark for y in x]
            mark = np.reshape(mark, (-1, 4))
        elif indexMark > -1:
            #print("no end")
            for i in range(indexMark+1, len(text)):
                content = text.iloc[i]['text']
                temp = content.split(',')
                mark.append(temp)
            mark = [y for x in mark for y in x]
            mark = np.reshape(mark, (-1, 4))
    
        #s = pd.Series({'Student name':student[0], 'Student ID':student[1], 'Student course ID':student[2]})
        studentDf = pd.DataFrame(columns = ('Student name', 'Student ID', 'Student course ID'), data = [student])
        courseDf = pd.DataFrame(columns = ('Course', 'Credit points obtained', 'Major(s)'), data = [course])
        marksDf = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'), data = mark)
    
        return studentDf, courseDf, marksDf
    

In [37]:
def convert_to_df2(text):
    result = []
    mark = []
    indexResult = 0
    indexMark = 0
    for i in range(len(text)-1):
        content = text.iloc[i]['text']
        temp = content.split(',')
        if 'Credit points gained' in content:
            indexResult = i+1
            break
        elif temp[0] == 'Year':
            #print("2")
            indexMark = i+1
            break
    
    if indexResult > 0:
        if indexResult != 1:
            for i in range(0,indexResult-1): 
                content = text.iloc[i]['text']
                temp = content.split(',')
                temp1 = temp[0].split(' ')
                result.append(temp1)
                result.append(temp[1:])
            result = [y for x in result for y in x]
            result = np.reshape(result, (-1, 7))
        for i in range(indexResult+2, len(text)-1):
            #print(indexResult)
            content = text.iloc[i]['text']
            temp = content.split(',')
            mark.append(temp)
    
    if indexMark > 0 :
        for i in range(indexMark, len(text)-1):
            content = text.iloc[i]['text']
            temp = content.split(',')
            #print(temp)
            mark.append(temp)
    elif indexMark == 0 and indexResult == 0:
        for i in range(len(text)-1):
            content = text.iloc[i]['text']
            temp = content.split(',')
            #print(temp)
            mark.append(temp)
        
    mark = [y for x in mark for y in x]
    #print(mark)
    mark = np.reshape(mark, (-1, 4))
        
            
    resultDf = pd.DataFrame(columns = ('Year', 'Session', 'Unit of study code', 'Unit of study name', 'Mark', 'Grade', 'Credit points'), data = result)
    marksDf2 = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'), data = mark)
        
    return resultDf, marksDf2
    

In [38]:
from tabula import wrapper

# extract multiple from all pages
multi_tables = wrapper.read_pdf("Transcript0903-2.pdf", pages="all", multiple_tables=True)

In [39]:
resultDf1 = pd.DataFrame(multi_tables[0]) 
resultDf1.duplicated()  #check whole dataframe if there is duplicated value
resultDf1 = resultDf1.drop_duplicates() 
resultDf1.columns = resultDf1.iloc[0]
resultDf1 = resultDf1.reindex(resultDf1.index.drop(0))


In [65]:
fp = open('Transcript.pdf', 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

textDf = pd.DataFrame(columns=['x1', 'y1', 'x2', 'y2', 'text'])
marksDf2 = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'))
resultDf2 = pd.DataFrame(columns = ('Year', 'Session', 'Unit of study code', 'Unit of study name', 'Mark', 'Grade', 'Credit points'))

page_no = 1

for page in PDFPage.create_pages(doc):
    
    interpreter.process_page(page)
    layout = device.get_result()
    textDf = parse_layout(layout, 0.25, 30)
    textDf['text'] = textDf['text'].apply(lambda x:','+ x)
    textDf1 = textDf.groupby(by='y')['text'].sum().reset_index()
    textDf1 = textDf1.sort_values('y', ascending=False)
    textDf1['text'] = textDf1['text'].apply(lambda x:x[1:])
    print(textDf1)

    if page_no == 1:
        studentDf, courseDf, marksDf = convert_to_df(textDf1)
    elif page_no == 2:
        resultDf2, marksDf2 = convert_to_df2(textDf1)
        
    page_no += 1
    
#print(marksDf)
#print(marksDf2)
#print(resultDf2)

#if marksDf2.empty == False:
marksDf = pd.concat([marksDf, marksDf2])
resultDf = pd.concat([resultDf1, resultDf2])

print(studentDf)
print(courseDf)
print(marksDf)
resultDf

        y                                               text
39  716.5                              Course result history
38  677.0  The details listed below are for the course Ba...
37  664.5  a copy of your online result history and not a...
36  645.0                      Student name Jeremiah Simmons
35  632.5                               Student ID 103059683
34  620.5                      Student course ID 103059683/1
33  595.5                                     Course details
32  569.5              Course,Bachelor of Advanced Computing
31  554.0                          Credit points obtained 48
30  539.0                          Major(s),Computer Science
29  520.5                              Unit of Study Results
28  494.0  Year Session Unit of study code,Unit of study ...
27  477.0  2017 S1C,COMP2022,Formal Languages and Logic,1...
26  462.0  2017 S1C,COMP2129,Operating Systems and Machin...
25  446.5  2017 S1C,INFO2820,Database Systems 1 (Advanced...
24  431.5  2017 S1C,MATH

,Year,Session,Unit of study code,Unit of study name,Mark,Grade,Credit points
1,2017,S1C,COMP2022,Formal Languages and Logic,100.0,HD,6
2,2017,S1C,COMP2129,Operating Systems and Machine Principles,100.0,HD,6
3,2017,S1C,INFO2820,Database Systems 1 (Advanced),100.0,HD,6
4,2017,S1C,MATH2069,Discrete Mathematics and GraphTheory,100.0,HD,6
5,2017,S2C,COMP2007,Algorithms and Complexity,100.0,HD,6
6,2017,S2C,COMP2121,Distributed Systems and Network Principles,100.0,HD,6
7,2017,S2C,INFO2110,Systems Analysis andModelling,100.0,HD,6
9,2017,S2C,INFO3404,Database Systems 2,100.0,HD,6
10,2018,S1C,COMP3308,Introduction to Artificial Intelligence,100.0,HD,6
11,2018,S1C,ENGG3800,Industry and Community Projects,100.0,HD,6


In [ ]:
    def getValueByKey(self, pages, interpreter, device, keyWord):
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            for x in layout:
                    if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
                    
                        rowIndex = 0
                        columnIndex = 0
                        for y in x:
                            if(y.get_text() == keyWord + '\n'):
                                rowIndex = y.bbox[0]
                                columnIndex = y.bbox[1]
                                break
                        
                        if(rowIndex != 0 and columnIndex != 0):
                            for x1 in layout:
                                if isinstance(x1, LTTextBox) or isinstance(x1, LTTextLine):
                                    for y1 in x1:
                                        if(y1.bbox[1] == columnIndex and y1.bbox[0] != rowIndex):
                                            return y1.get_text().strip()

    def getValueBySplit(self, pages, interpreter, device, keyWord):
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            for x in layout:
                 if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
                        for y in x:
                            if (keyWord in y.get_text()):
                                return y.get_text().strip('\n').split(keyWord)[1]
                            
    def getAverageByLocation(self, pages, interpreter,device, keyWord):
#         print(dir(pages))
        format_years = []
        format_semesters = []
        format_sams = []
        format_aams = []
        n = 0
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            count = 0
            if n == 1:
                for x in layout:
                    if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
#                         print(x.get_text())
                        if count == 0:
                            years = x.get_text().split('\n')

                            for year in years:
                                if(year != 'Year' and year != ''):
                                    format_years.append(year)
#                             print(format_years)
                            count += 1
                        elif count == 1:
                            semesters = x.get_text().split('\n')
                            for semester in semesters:
                                if(semester != 'Session' and semester != ''):
                                    format_semesters.append(semester)
                            count += 1
                        elif count == 2:
                            sams = x.get_text().split('\n')
                            for sam in sams:
                                if(sam != ''):
                                    format_sams.append(sam.replace('\n', ''))
                            count += 1
                        elif count == 3:
                            aams = x.get_text().split('\n')
                            for aam in aams:
                                if(aam != ''):
                                    format_aams.append(aam.replace('\n', ''))
                            count += 1
                a = pd.DataFrame({'Year': format_years, 'Session': format_semesters, 'Semester average mark': format_sams, 'Annual average mark': format_aams})
                return a
            n += 1
        
            for x in layout:
                if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):

                    rowIndex = 0
                    columnIndex = 0
                    for y in x:
                        if(y.get_text() == keyWord + '\n'):
                            rowIndex = y.bbox[0]
                            columnIndex = y.bbox[1]
                            break

                    if(rowIndex != 0 and columnIndex != 0):
                        format_years = []
                        format_semesters = []
                        format_sams = []
                        format_aams = []
                        index = 0

                        for x1 in layout:
                            if isinstance(x1, LTTextBox) or isinstance(x1, LTTextLine):
                                if(x1.bbox[1] < columnIndex):
#                                     print(x1.get_text())
                                    if(index == 0):
                                        years = x1.get_text().split('\n')
                                        for year in years:
                                            if(year != 'Year' and year != ''):
                                                format_years.append(year)
                        #page1
#                                     if(index == 1):
#                                         semesters = x1.get_text().split('\n')
#                                         for semester in semesters:
#                                             if(semester != 'Session' and semester != ''):
#                                                 format_semesters.append(semester)
                                                    
#                                     if(index == 4):
#                                         sams = x1.get_text().split('\n')
#                                         for sam in sams:
#                                             if(sam != ''):
#                                                 format_sams.append(sam.replace('\n', ''))
                                                
#                                     if(index == 5):
#                                         aams = x1.get_text().split('\n')
#                                         for aam in aams:
#                                             if(aam != ''):
#                                                 format_aams.append(aam.replace('\n', ''))
                         #page2
                                    if(index == 2):
                                        semesters = x1.get_text().split('\n')
                                        for semester in semesters:
                                            if(semester != 'Session' and semester != ''):
                                                format_semesters.append(semester)

                                    if(index == 5):
                                        sams = x1.get_text().split('\n')
                                        for sam in sams:
                                            if(sam != ''):
                                                format_sams.append(sam.replace('\n', ''))

                                    if(index == 6):
                                        aams = x1.get_text().split('\n')
                                        for aam in aams:
                                            if(aam != ''):
                                                format_aams.append(aam.replace('\n', ''))

                                    index = index + 1

        a = pd.DataFrame({'Year': format_years, 'Session': format_semesters, 'Semester average mark': format_sams, 'Annual average mark': format_aams})
        return a

In [ ]:
import pandas as pd
from tabula import wrapper

path = 'DemoTranscript.pdf'

file = open(path, 'rb')
praser = PDFParser(file)
doc = PDFDocument(praser, password='')
if not doc.is_extractable:
    raise PDFTextExtractionNotAllowe
rsrcmgr = PDFResourceManager(caching = False)
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)


In [ ]:
Student_name = []
Student_name.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Student name '))
Student_id = []
Student_id.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Student ID '))
Student_course_id = []
Student_course_id.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Student course ID '))
student = [Student_name + Student_id + Student_course_id]

student_info = pd.DataFrame(columns = ('Studnet name', 'Student ID', 'Student course ID'), data = student)
# student_info = pd.DataFrame({'Studnet name': Student_name, 'Student ID': Student_id, 'Student course ID': Student_course_id})
        
print("-------This student information-------")
student_info
# student_info.sort_index(axis=1, ascending=False)



In [ ]:
value_course = []
value_course.append(pdf2TxtManager.getValueByKey(PDFPage.create_pages(doc), interpreter, device, 'Course'))
value_major = []
value_major.append(pdf2TxtManager.getValueByKey(PDFPage.create_pages(doc), interpreter, device, 'Major(s)'))
Credit_points_obtained = []
Credit_points_obtained.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Credit points obtained '))
courseDetails = [value_course + value_major + Credit_points_obtained]

course_details = pd.DataFrame(columns = ('Course', 'Major(s)', 'Credit points obtained'), data = courseDetails)
# course_details = pd.DataFrame(columns = {'Course', 'Major(s)', 'Credit points obtained'}, data = courseDetails)
# course_details = pd.DataFrame({'Course': value_course, 'Credit points obtained': Credit_points_obtained, 'Major(s)': value_major})

print("\n-------This course details-------")
course_details

In [ ]:
df = wrapper.read_pdf('DemoTranscript.pdf')
df

In [ ]:



# list = pdf2TxtManager.getAverageByLocation(PDFPage.create_pages(doc), interpreter, device, 'Your Average Marks')
# print("\n-------This average marks-------")
# list.sort_index(axis=1, ascending=False)
list = pdf2TxtManager.getAverageByLocation(PDFPage.create_pages(doc), interpreter, device, 'Your Average Marks')
print("\n-------This average marks-------")
list.sort_index(axis=1, ascending=False)


In [ ]:
path = 'Demotranscript.pdf'

file = open(path, 'rb')
praser = PDFParser(file)
doc = PDFDocument(praser, password='')
if not doc.is_extractable:
    raise PDFTextExtractionNotAllowe
rsrcmgr = PDFResourceManager(caching = False)
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pages = PDFPage.create_pages(doc)

text  = []
y_axis = []
x_axis = []

data = pd.DataFrame(columns = ('y-axis', 'x-axis', 'text'))
for page in pages:
    interpreter.process_page(page)
    layout = device.get_result()
    for x in layout:
            if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
                for y in x:
                    text.append(y.get_text())
                    y_axis.append(int(y.bbox[1]))
                    x_axis.append(round(y.bbox[0], 2))

data['y-axis'] = y_axis
data['text'] = text
data['x-axis'] = x_axis

# data = data.sort_values(by = 'y-axis')

student_detail = pd.DataFrame(columns = ('Student name', 'Student ID', 'Student course ID'))
course_detail = pd.DataFrame(columns = ('Course', 'Credit points obtained', 'Major(s)'))
average_mark = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'))

course_row = 0
course_column = 0
major_row = 0
major = column = 0

for index, row in data.iterrows():
    if 'Student name' in row['text']:
        student_detail['Student name'] = [row['text'].strip('\n').split('Student name ')[1]]
    if 'Student ID' in row['text']:
        student_detail['Student ID'] = [row['text'].strip('\n').split('Student ID ')[1]]
    if 'Student course ID' in row['text']:
        student_detail['Student course ID'] = [row['text'].strip('\n').split('Student course ID ')[1]]
    if 'Credit points obtained' in row['text']:
        course_detail['Credit points obtained'] = [row['text'].strip('\n').split('Credit points obtained ')[1]]

        

def getCourseDetail(data, keyWord, course_detail):
    rowIndex = 0
    columnIndex = 0
    keyWord_1 = keyWord + '\n'
    
    for index, row in data.iterrows():
        if keyWord_1 in row['text']:
            columnIndex = row['y-axis']
            rowIndex = row['x-axis']

    for index, row in data.iterrows():
        if row['y-axis'] == columnIndex and row['x-axis'] != rowIndex:
            course_detail[keyWord] = [row['text'].strip('\n')]
            
getCourseDetail(data, 'Course', course_detail)
getCourseDetail(data, 'Major(s)', course_detail)

def getAverageMark(data, keyWord, average_mark):
    index = 0
    columnIndex = 0
    for index, row in data.iterrows():
        if keyWord in row['text']:
            columnIndex = row['y-axis']
    
    for index, row in data.iterrows():
        if row['y-axis'] < columnIndex and 'Year' in row['text']:
            index += 1
            break
        elif row['y-axis'] < columnIndex and index < 2:
            average_mark['Year'] = [row['text'].strip('\n')]
        elif row['y-axis'] < columnIndex and 'Session' in row['text']:
            index += 1
            break
        elif row['y-axis'] < columnIndex and index < 3:
            average_mark['Session'] = [row['text'].strip('\n')] 
        elif row['y-axis'] < columnIndex and 'Semester average mark' in row['text']:
            index += 1
            break        
        elif row['y-axis'] < columnIndex and index < 4:
            average_mark['Session'] = [row['text'].strip('\n')]         
            
            

getAverageMark(data, 'Your Average Marks', average_mark)       
        


data



In [ ]:
student_detail

In [ ]:
course_detail